In [40]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

import numpy

import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
#from imblearn.over_sampling import SMOTE, ADASYN
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna

import math

from sklearn.metrics import roc_curve, auc 

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, SVMSMOTE, KMeansSMOTE
from sklearn.model_selection import train_test_split

import xgboost as xgb
import catboost as cb
import shap

import glob
import pickle

import re

dir_path = '../data/PST/'


In [41]:
X_test = pd.read_pickle(dir_path + 'dataset_to_pred.pkl')

In [42]:
files = glob.glob(dir_path + '*')

In [43]:
def load_max_score_model(model_prefix, files):
    pattern = re.compile(rf"{model_prefix}_\d+_(\d+).pkl$")
    max_file = None
    max_score = -1
    for file in files:
        match = pattern.search(file)
        if match:
            score = int(match.group(1))
            if score > max_score:
                max_score = score
                max_file = file
    if max_file:
        with open(max_file, 'rb') as f:
            model = pickle.load(f)
        print(f"Loaded {model_prefix} model from {max_file}")
        return model
    else:
        print(f"No {model_prefix} model files found.")
        return None

lgbm_clf = load_max_score_model("lgbm_clf", files)
cat_clf = load_max_score_model("cat_clf", files)

Loaded lgbm_clf model from ../data/PST/lgbm_clf_98_8397.pkl
Loaded cat_clf model from ../data/PST/cat_clf_98_8397.pkl
Loaded xgb_clf model from ../data/PST/xgb_clf_98_8397.pkl


In [44]:

y_pred_proba_lgbm = lgbm_clf.predict_proba(X_test)[:, 1]
y_pred_proba_cat = cat_clf.predict_proba(X_test)[:, 1]

y_test = (y_pred_proba_lgbm+y_pred_proba_cat)/2



[LightGBM] [Warning] feature_fraction is set=0.41350902619079855, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41350902619079855
[LightGBM] [Warning] lambda_l2 is set=0.11494135705960554, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.11494135705960554
[LightGBM] [Warning] lambda_l1 is set=0.8775187936705214, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8775187936705214
[LightGBM] [Warning] bagging_fraction is set=0.6074378238140433, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6074378238140433
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


In [45]:
X_test = pd.read_pickle(dir_path + 'X_test.pkl')
df_test = pd.read_pickle(dir_path + 'df_test.pkl')
test_index_o = pd.read_pickle(dir_path + 'test_index_o.pkl')

In [46]:

test_index_o['score'] = y_test
pred_data_test = test_index_o

pred_data_test['ref_title'] = pred_data_test['ref_title'].str.lower()
pred_data_test['ref_title'] = pred_data_test['ref_title'].str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True)

pred_data_test.loc[pred_data_test['score'] <= 0,'score'] = 0
pred_data_test.loc[pred_data_test['score'] >= 1,'score'] = 1

pred_data_test.to_csv(dir_path + 'pred_test.csv',index = False)


In [48]:

with open(dir_path + 'valid_submission_test.json', 'r') as file:
    valid_submission_data = json.load(file)

with open(dir_path + 'pred_test.csv', 'r') as file:
    pred_data = pd.read_csv(file)

pred_data['ref_title'] = pred_data['ref_title'].str.lower().str.replace(' ', '')

new_valid_sub_target = {}

for paper_id, entries in valid_submission_data.items():
    new_scores = []
    for entry in entries:
        title = entry['title'].lower().replace(' ', '')
        #print(title,paper_id)
        if title:
            score = pred_data.loc[(pred_data['_id'] == paper_id) & (pred_data['ref_title'] == title), 'score'].max()
            if pd.isna(score):
                score = 0 
        else:
            score = 0 
        new_scores.append(score)
    new_valid_sub_target[paper_id] = new_scores

with open('new_test_sub_target.json', 'w') as file:
    json.dump(new_valid_sub_target, file)

  